In [ ]:
from teehr import Evaluation
from pathlib import Path

In [ ]:

# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "test_study")

In [ ]:
# Create an Evaluation object
ev = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
ev.enable_logging()

In [ ]:
from teehr.models.filters import FilterOperators as fop

In [ ]:

from teehr.models.filters import TimeseriesFilter
ts_fields = ev.primary_timeseries.field_enum()

In [ ]:
ts = ev.primary_timeseries.to_sdf()
ts.schema

In [ ]:
# Get the timeseries data
ts_df = ev.primary_timeseries.query(
    filters=[
        TimeseriesFilter(
            column=ts_fields.location_id,
            operator=fop.eq,
            value="gage-A"
        )
    ]
).to_pandas()
ts_df.head()

In [ ]:
from teehr.models.filters import JoinedTimeseriesFilter
jt_fields = ev.joined_timeseries.field_enum()

In [ ]:
jts_df = ev.joined_timeseries.query(
    filters=[
        JoinedTimeseriesFilter(
            column=jt_fields.primary_location_id,
            operator=fop.eq,
            value="gage-A"
        )
    ]
).to_pandas()
jts_df.head()

In [ ]:
from teehr import Metrics as metrics

In [ ]:
metrics.KlingGuptaEfficiency()

In [ ]:
metrics_df = ev.metrics.query(
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency()
    ]
).to_pandas()
metrics_df.head()

In [ ]:
metrics_df = ev.metrics.query(
    filters=[
        JoinedTimeseriesFilter(
            column=jt_fields.primary_location_id,
            operator=fop.eq,
            value="gage-A"
        )
    ],
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency()
    ]
).to_geopandas()
metrics_df.head()

In [ ]:
metrics_df = ev.metrics.query(
    filters=[
        "primary_location_id = 'gage-A'"
    ],
    group_by=["primary_location_id"],
    order_by=["primary_location_id"],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency()
    ]
).to_pandas()
metrics_df.head()

In [ ]:
kge = metrics.KlingGuptaEfficiency()
kge.output_field_name = "KGE"

In [ ]:
metrics_df = ev.metrics.query(
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        kge,
        metrics.NashSutcliffeEfficiency(),
        metrics.Average()
    ]
).to_pandas()
metrics_df.head()

In [ ]:
pa = metrics.Average()
pa.input_field_names = [jt_fields.primary_normalized_flow]
pa.output_field_name = "normalized_primary_avg"

In [ ]:
metrics_df = ev.metrics.query(
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency(),
        pa
    ]
).to_pandas()
metrics_df.head()

In [ ]:
kge = metrics.KlingGuptaEfficiency()
kge

In [ ]:
# Estimate uncertainty of the KGE metric.
from teehr.models.metrics.bootstrap_models import Bootstrappers

In [ ]:
# Define a bootstrapper with custom parameters.
boot = Bootstrappers.CircularBlock(
    seed=50,
    reps=500,
    block_size=10,
    quantiles=[0.05, 0.95]
)
kge = metrics.KlingGuptaEfficiency(bootstrap=boot)
kge.output_field_name = "BS_KGE"

include_metrics = [kge, metrics.KlingGuptaEfficiency()]

In [ ]:
# Get the currently available fields to use in the query.
flds = ev.joined_timeseries.field_enum()

metrics_df = ev.metrics.query(
    include_metrics=include_metrics,
    group_by=[flds.primary_location_id],
    order_by=[flds.primary_location_id]
).to_geopandas()
metrics_df